In [1]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


In [5]:
from utils import ring, connection, pipe, cutout, two_sided_connection
from solid import cylinder, translate, rotate, cube
from solid.utils import *
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def bottom_grid(d=60, w=2, hole_size=6, hole_gap=2):
    plate = cylinder(h=w, d=d, segments=100)
    hole = cylinder(h=w, d=hole_size, segments=10)
    pane = range(-d//2, d//2, hole_size+hole_gap)
    transition = ring(d1=d, h=w, w=w*2)

    holes = [translate((x, y, 0))(hole) for x in pane for y in pane]

    return plate - holes + transition


def spin_around_tower(obj, count=4, alpha=45, h=160):
    angles = range(180, 540, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def cup_con(d1=52, d2=60, w=2, height=80, inlet_delta=5):
    c = d2
    gradient = (d2-d1)/height
    inlet = (gradient*inlet_delta+w) * 2
    socket_d1_delta = d1 + inlet
    socket_d2_delta = d2 + inlet
    c = cutout(c)

    base = ring(d1=socket_d1_delta, d2=socket_d2_delta, h=height, w=w)
    socket = base - c

    cup = ring(d1=d1, d2=d2, w=w, h=height)
    cup += bottom_grid(d=d1, w=w)

    outline = cylinder(d1=d1, d2=d2, h=height, segments=100)

    return (socket, cup, outline)


def aero_tank(d=180, height=180, wall=2, socket_h=20, tooth=4):
    body = ring(d1=d, h=height, w=wall)
    bottom = cylinder(d=d, h=wall, segments=100)
    (_, socket) = connection(d=d, h=socket_h,
                             wall=wall, tooth_depth=tooth, tooth_height=tooth*2)

    return bottom + body + up(height-socket_h)(socket)


def aero_tower(w=1, h=160, d=160, socket_h=20, pipe_count=1, alpha=45, cup_d=60, cup_h=80, tooth=4):
    aero_tower = pipe(d=d, h=h, w=w, tooth=tooth, socket_h=socket_h)
    (cup_s, cup, cup_outline) = cup_con(d1=cup_d-cup_h //
                                        10, d2=cup_d, w=w, height=cup_h, inlet_delta=5)
    outline = cylinder(d=d, h=d, segments=100)
    holes = spin_around_tower(translate((0, 0, h/2-4))
                              (cup_outline), count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(
        translate((0, 0, h/2+w*5))(cup_s), h=d, count=pipe_count, alpha=alpha)

    holes = translate((0, 0, -h/2))(holes)
    sockets = translate((0, 0, -h/2))(sockets)
    aero_tower = aero_tower - holes + sockets
    aero_tower *= outline

    tank = aero_tank(d=180, height=180, wall=2, tooth=tooth)
    adapter = two_sided_connection(outer_d=180, outer_w=2, outer_tooth=tooth,
                                   height=socket_h, inner_d=d, inner_w=w, inner_tooth=tooth)\
        + cylinder(r=90-2-tooth-0.2, h=w, segments=100)\
        - cylinder(d=6, h=w)

    return (aero_tower, cup, tank, adapter)


(tower, cup, tank, adapter) = aero_tower(pipe_count=5)
render(tower, outfile='./outfiles/aero_tower.stl')
render(cup, outfile='./outfiles/plant_cup.stl')
render(tank, outfile='./outfiles/aero_tower_tank.stl')
render(adapter, outfile='./outfiles/aero_tower_tank_adapter.stl')


Geometries in cache: 28
Geometry cache size in bytes: 1249088
CGAL Polyhedrons in cache: 4
CGAL cache size in bytes: 100754544
Total rendering time: 0:03:40.718
   Top level object is a 3D object:
   Simple:         no
   Vertices:    21429
   Halfedges:   73150
   Edges:       36575
   Halffacets:  29046
   Facets:      14523
   Volumes:         3
EXPORT-WARNING: Exported object may not be a valid 2-manifold and may need repair


Geometries in cache: 54
Geometry cache size in bytes: 147408
CGAL Polyhedrons in cache: 7
CGAL cache size in bytes: 6617776
Total rendering time: 0:00:22.606
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     1222
   Halfedges:    3954
   Edges:        1977
   Halffacets:   1516
   Facets:        758
   Volumes:         2


Geometries in cache: 7
Geometry cache size in bytes: 491384
CGAL Polyhedrons in cache: 8
CGAL cache size in bytes: 39119712
Total rendering time: 0:00:44.157
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     5661
   Halfedges:   19524
   Edges:        9762
   Halffacets:   7742
   Facets:       3871
   Volumes:         2


Geometries in cache: 11
Geometry cache size in bytes: 965032
CGAL Polyhedrons in cache: 12
CGAL cache size in bytes: 83080232
Total rendering time: 0:00:57.874
   Top level object is a 3D object:
   Simple:         no
   Vertices:    14565
   Halfedges:   51354
   Edges:       25677
   Halffacets:  21182
   Facets:      10591
   Volumes:         2
EXPORT-WARNING: Exported object may not be a valid 2-manifold and may need repair


In [7]:

from utils import pipe, ring, connection, mount_to_bottom, tended_obj, cutout, sliding_connector
from solid import cylinder, translate, rotate, cube, hull
from solid.utils import *
from viewscad import Renderer
from math import sqrt

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def tower_head(d=160, w=2, tooth=4, socket_h=20, h=100, gap=0.1):
    delta_r = w + tooth + gap
    base_d = d-delta_r*2
    filter_d = 8
    filter_l = 100
    piezo_d = 20
    piezo_h = 3
    slider_l = 30
    slider_w = 20
    pump_d = 27.4
    pump_hole_delta = -((pump_d + slider_l)/2 + w*2)

    # head body
    body = pipe(d=d, w=w, tooth=tooth, socket_h=socket_h, h=h)

    base = cylinder(d=base_d, h=w, segments=100)
    (slider_s, slider_a) = sliding_connector(
        length=slider_l, width=slider_w, gap=0.05)

    cable_hole = rotate(90)(
        translate((-d/2, 0, h/2))(
            rotate((0, 90, 0))(
                cylinder(d=4, h=2*w, segments=100)
            )
        )
    )

    pump_hole = translate((pump_hole_delta, 0, 0))(
            cylinder(d=pump_d, h=20, segments=100))
    

    head = body + base - cable_hole
    head -= translate((-slider_l/2, 0, 0))(pump_hole + slider_s)

    # fogger module

    fogger = down(piezo_h/2)(
        cube((slider_l, piezo_d+4, piezo_d+piezo_h+4), center=True)
        + translate((-slider_l/2, 0, (piezo_d+piezo_h+4)/2))(slider_a)
    )

    fogger += right((slider_l+piezo_h)/2)(rotate((0, 90, 0))(
        hull()(
            cylinder(d=piezo_d+4, h=piezo_h+2, center=True, segments=100),
            translate((piezo_h, 0, -(piezo_h)/2))(
                cylinder(d=piezo_d+4, h=.1, center=True, segments=100))
        )
        - cylinder(d=piezo_d, h=piezo_h, center=True, segments=100)
        - cylinder(d=piezo_d-4, h=piezo_h+2, center=True, segments=100)
        - translate((-2*piezo_d, -piezo_d, -piezo_h)
                    )(cube((piezo_d*2, piezo_d*2, piezo_h*2)))
    ))

    fogger -= down(4)(rotate((0, 80, 0))(
        cylinder(d=filter_d, h=filter_l, center=True, segments=100)
    ))

    transition_delta = 4
    tank_width_addon = 10
    fogger += translate((-(slider_l+pump_d/2+tank_width_addon)/2, 0, -piezo_h))(
        cube((pump_d/2+tank_width_addon, piezo_d+transition_delta, piezo_d+transition_delta), center=True)
        - translate((1, 0, 2))(
            cube((pump_d/2-2+tank_width_addon, piezo_d, piezo_d+2), center=True)
        )
        - translate((0, 0, piezo_d*0.75))(
            cube((pump_d/2+tank_width_addon, piezo_d+transition_delta, piezo_d+2), center=True)
        )
    )

    cable_s = 2
    cable_rail = cube((slider_l/2+cable_s, cable_s, slider_w))
    fogger -= translate((-cable_s, -1, piezo_d/2))(cable_rail)

    # head lid

    head_lid_adapter_inner_d = d-(w+gap+tooth)*2
    head_lid_body = cylinder(d=head_lid_adapter_inner_d, segments=100, h=w)
    handle_base_thickness = 10
    handle_top_thickness = handle_base_thickness + (socket_h-w)*2

    (head_lid_adapter, _) = connection(d=d, h=socket_h,
                                       wall=w, tooth_depth=tooth, tooth_height=2*tooth, gap=0.2)

    head_lid_body += hull()(
        up(w/2)(cube((handle_base_thickness, head_lid_adapter_inner_d, w), center=True)),
        up((socket_h-w/2))(cube((handle_top_thickness,
                                 head_lid_adapter_inner_d, w), center=True)),
    ) * cylinder(d=head_lid_adapter_inner_d, segments=100, h=socket_h)

    head_lid = head_lid_adapter + head_lid_body

    return (head, fogger, head_lid)


(head, fogger, head_lid) = tower_head()

render(head, outfile='outfiles/aero_tower_head.stl')
render(fogger, outfile='outfiles/aero_tower_head_fogger.stl')
render(head_lid, outfile='outfiles/aero_tower_head_lid.stl')


Geometries in cache: 21
Geometry cache size in bytes: 208968
CGAL Polyhedrons in cache: 22
CGAL cache size in bytes: 15308744
Total rendering time: 0:00:23.583
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     2449
   Halfedges:    7958
   Edges:        3979
   Halffacets:   2748
   Facets:       1374
   Volumes:         3


Geometries in cache: 24
Geometry cache size in bytes: 170544
CGAL Polyhedrons in cache: 15
CGAL cache size in bytes: 5858336
Total rendering time: 0:00:06.350
   Top level object is a 3D object:
   Simple:        yes
   Vertices:      735
   Halfedges:    2258
   Edges:        1129
   Halffacets:    796
   Facets:        398
   Volumes:         3


Geometries in cache: 11
Geometry cache size in bytes: 77992
CGAL Polyhedrons in cache: 7
CGAL cache size in bytes: 4153952
Total rendering time: 0:00:07.833
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     1302
   Halfedges:    4086
   Edges:        2043
   Halffacets:   1358
   Facets:        679
   Volumes:         3
